In [1]:
from bs4 import BeautifulSoup
import requests
import json
import ast
import sys
sys.path.insert(1,r'C:\Users\samga\OneDrive\Desktop\Charts')
#sys.path.insert(1,r'C:\Users\Owner\OneDrive\Desktop\Charts')

import SGCharts

from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import datetime
import pickle

C:\Users\samga\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Scraping CoinMarket Cap

In [15]:
date = '{yyyy}{mm}{dd}'.format(yyyy='2020',mm='07',dd='20')
url = 'https://coinmarketcap.com/historical/{date}/'.format(date=date)
response = requests.get(url)

soup = BeautifulSoup(response.content,'lxml')
a= soup.find_all()

# Getting Data We Want

In [29]:

coin = [i.text for i in soup.find_all('a',class_='cmc-table__column-name--symbol cmc-link')]
market_cap = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap')]
price = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price')]
volume = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__volume-24-h')]
supply = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply')]
supply = [i.split(' ')[0] for i in supply]
day = [date]*len(coin)

# Sample Day of Data

In [30]:
pd.DataFrame({'yyyymmdd':day,'Coin':coin,'Market Cap':market_cap,'Price':price,'Volume':volume,'Supply':supply} )

,yyyymmdd,Coin,Market Cap,Price,Volume,Supply
0,20200720,BTC,"$168,970,262,858.50","$9,164.23","$13,755,604,145.72","18,438,018"
1,20200720,ETH,"$26,411,853,755.56",$236.15,"$5,600,686,192.18","111,842,047"
2,20200720,USDT,"$9,193,521,415.15",$1.00,"$16,920,167,823.51","9,187,991,663"
3,20200720,XRP,"$8,741,285,749.43",$0.1951,"$848,780,530.81","44,812,133,482"
4,20200720,BCH,"$4,113,723,679.68",$222.75,"$952,296,137.79","18,467,538"
5,20200720,BSV,"$3,127,401,196.08",$169.36,"$1,055,569,739.10","18,466,008"
6,20200720,ADA,"$3,057,429,414.70",$0.1179,"$176,280,625.60","25,927,070,538"
7,20200720,LTC,"$2,731,558,150.16",$41.98,"$1,152,077,278.88","65,074,579"
8,20200720,BNB,"$2,719,514,135.88",$17.48,"$203,143,854.50","155,536,713"
9,20200720,CRO,"$2,590,759,653.05",$0.1418,"$64,769,273.49","18,268,949,772"


# Scrape Over Long Period of Time

In [82]:
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2022, 10, 30)

days = (end-start).days

for i in tqdm(range(0,days)):
    date = (start+datetime.timedelta(days=i))#.strftime('%Y%m%d')
    if date.weekday() not in [5,6]:
        
        day = date.strftime('%Y%m%d')
        #date = '{yyyy}{mm}{dd}'.format(yyyy='2020',mm='07',dd='20')
        url = 'https://coinmarketcap.com/historical/{day}/'.format(day=day)
        response = requests.get(url)

        soup = BeautifulSoup(response.content,'lxml')
        a= soup.find_all()
        
        coin = [i.text for i in soup.find_all('a',class_='cmc-table__column-name--symbol cmc-link')]
        market_cap = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap')]
        price = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price')]
        volume = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__volume-24-h')]
        supply = [i.text for i in soup.find_all('td',class_='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply')]
        supply = [i.split(' ')[0] for i in supply]
        days = [day]*len(coin)
        df = pd.DataFrame({'yyyymmdd':days,'Coin':coin,'Market Cap':market_cap,'Price':price,'Volume':volume,'Supply':supply} )
        df.to_csv('crypto_data/{date}'.format(date=day))
        
        #print('Weekend')
        #print(date.strftime('%Y%m%d'))
        #pass
    #else:
    #    print(date.strftime('%Y%m%d'))
    #print(date)
    #print(date.strftime('%Y%m%d'))
    #print(date.weekday())

100%|██████████████████████████████████████████████████████████████████████████████| 2128/2128 [05:39<00:00,  6.27it/s]
